In [ ]:
from src.pdfinsight import pdf_extractor, remove_toc, combine_categories, pivot_df_by_heading, df2docstore
import pandas as pd
import os

In [ ]:
df_combined = pd.DataFrame()
for (root,dirs,files) in os.walk('.', topdown=True):
    for file in files:
        # only use it on pdf docs. ignore other file type such as txt
        if file.split(".")[-1] == "pdf":
            # print(root+"\\"+file)
            df = pdf_extractor(root+"\\"+file, toc_pages = 1, precision_dp = 2)
            df_combined = pd.concat([df_combined,df])
    break

# reset index
df_combined.reset_index(drop=True, inplace=True)

In [ ]:
df_combined

In [ ]:
# we check the ymin difference between each row INSTEAD OF BLOCK. 
# this is important in determining the para_thres
# text below the para_thres would be considered to be in the same block of text (ie para) 
# seems like if those between 0 to 20 is above 70%, we can set that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['ymin_round'].diff()], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

In [ ]:
# double check the ymin_thres between REFINED_BLOCKS
# as long as our defined para_thres of 20 is the majority
# then should be safe to use that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['block_ymin_diff']], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

In [ ]:
df_combined = remove_toc(df_combined)
df_combined

In [ ]:
transformed_df = pivot_df_by_heading(df_combined)

In [ ]:
transformed_df

In [ ]:
link_dict = dict(zip(transformed_df.file.unique(), transformed_df.file.unique()))

In [ ]:
df2docstore(transformed_df, chunk_size = 1000, link_dict = link_dict)